# ⚡ Latensbenchmark - Förstå modellens prestanda

**Mål**: Förstå hur vi mäter och tolkar modellens latens (svarstid).

I detta notebook kommer vi att:
- Förstå vad latens är och varför det är viktigt
- Se hur benchmark fungerar (warmup, runs, providers)
- Tolka resultat (p50, p95, histogram)
- Experimentera med olika inställningar

> **💡 Tips**: Latens är avgörande för edge deployment - en modell som är för långsam är inte användbar i verkligheten!


## 🤔 Vad är latens och varför är det viktigt?

**Latens** = tiden det tar för modellen att göra en förutsägelse (inference time).

**Varför viktigt för edge**:
- **Realtidsapplikationer** - robotar, autonoma fordon
- **Användarupplevelse** - ingen vill vänta 5 sekunder på en bildklassificering
- **Resursbegränsningar** - Raspberry Pi har begränsad CPU/memory

<details>
<summary>🔍 Klicka för att se typiska latensmål</summary>

**Typiska latensmål**:
- **< 10ms**: Realtidsvideo, gaming
- **< 100ms**: Interaktiva applikationer
- **< 1000ms**: Batch processing, offline analys

**Vår modell**: Förväntar oss ~1-10ms på CPU (bra för edge!)

</details>


## 🔧 Hur fungerar benchmark?

**Benchmark-processen**:
1. **Warmup** - kör modellen några gånger för att "värmma upp" (JIT compilation, cache)
2. **Runs** - mäter latens för många körningar
3. **Statistik** - beräknar p50, p95, mean, std

**Varför warmup?**
- Första körningen är ofta långsam (JIT compilation)
- Cache-värme påverkar prestanda
- Vi vill mäta "steady state" prestanda


In [ ]:
# Kör benchmark med olika inställningar
print("🚀 Kör benchmark...")

# Använd modellen från föregående notebook (eller skapa en snabb)
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 256 --output-dir ./models_bench


In [ ]:
# Benchmark med olika antal runs för att se varians
import os

# Test 1: Få runs (snabb)
print("📊 Test 1: 10 runs")
!python -m piedge_edukit.benchmark --fakedata --model-path ./models_bench/model.onnx --warmup 3 --runs 10 --providers CPUExecutionProvider


In [ ]:
# Visa benchmark-resultat
if os.path.exists("./reports/latency_summary.txt"):
    with open("./reports/latency_summary.txt", "r") as f:
        print("📈 Benchmark-resultat:")
        print(f.read())
else:
    print("❌ Benchmark-rapport saknas")


In [ ]:
# Läs detaljerade latensdata och visualisera
import pandas as pd
import matplotlib.pyplot as plt

if os.path.exists("./reports/latency.csv"):
    df = pd.read_csv("./reports/latency.csv")
    
    print(f"📊 Latens-statistik:")
    print(f"Antal mätningar: {len(df)}")
    print(f"Mean: {df['latency_ms'].mean():.2f} ms")
    print(f"Std: {df['latency_ms'].std():.2f} ms")
    print(f"Min: {df['latency_ms'].min():.2f} ms")
    print(f"Max: {df['latency_ms'].max():.2f} ms")
    
    # Histogram
    plt.figure(figsize=(10, 6))
    plt.hist(df['latency_ms'], bins=20, alpha=0.7, edgecolor='black')
    plt.xlabel('Latens (ms)')
    plt.ylabel('Antal')
    plt.title('Latens-distribution')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Box plot
    plt.figure(figsize=(8, 6))
    plt.boxplot(df['latency_ms'])
    plt.ylabel('Latens (ms)')
    plt.title('Latens Box Plot')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("❌ Latens CSV saknas")


## 🤔 Reflektionsfrågor

<details>
<summary>💭 Varför är p95 viktigare än mean för edge deployment?</summary>

**Svar**: p95 (95:e percentilen) visar den värsta latensen som 95% av användarna upplever. Det är viktigare än mean eftersom:

- **Användarupplevelse**: En användare som får 100ms latens kommer att märka det, även om mean är 10ms
- **SLA-krav**: Många system har SLA-krav på p95 latens
- **Outliers**: Mean kan påverkas av extrema värden, p95 är mer robust

</details>

<details>
<summary>💭 Vad händer med latens-variansen när du ökar antal runs?</summary>

**Svar**: Med fler runs får vi:
- **Mer stabil statistik** - p50/p95 blir mer tillförlitliga
- **Bättre förståelse av varians** - ser om modellen är konsistent
- **Mindre påverkan av outliers** - enstaka långsamma körningar påverkar mindre

**Experiment**: Kör benchmark med 10, 50, 100 runs och jämför standardavvikelsen.

</details>


## 🎯 Ditt eget experiment

**Uppgift**: Kör benchmark med olika inställningar och jämför resultaten.

**Förslag**:
- Testa olika antal runs (10, 50, 100)
- Jämför warmup-effekten (0, 3, 10 warmup)
- Analysera variansen mellan körningar

**Kod att modifiera**:
```python
# Ändra dessa värden:
WARMUP_RUNS = 5
BENCHMARK_RUNS = 50

!python -m piedge_edukit.benchmark --fakedata --model-path ./models_bench/model.onnx --warmup {WARMUP_RUNS} --runs {BENCHMARK_RUNS} --providers CPUExecutionProvider
```


In [ ]:
# TODO: Implementera ditt experiment här
# Ändra värdena nedan och kör benchmark

WARMUP_RUNS = 5
BENCHMARK_RUNS = 50

print(f"🧪 Mitt experiment: warmup={WARMUP_RUNS}, runs={BENCHMARK_RUNS}")

# TODO: Kör benchmark med dina inställningar
# !python -m piedge_edukit.benchmark --fakedata --model-path ./models_bench/model.onnx --warmup {WARMUP_RUNS} --runs {BENCHMARK_RUNS} --providers CPUExecutionProvider


## 🎉 Sammanfattning

Du har nu lärt dig:
- Vad latens är och varför det är kritiskt för edge deployment
- Hur benchmark fungerar (warmup, runs, statistik)
- Hur man tolkar latens-resultat (p50, p95, varians)
- Varför p95 är viktigare än mean för användarupplevelse

**Nästa steg**: Gå till `03_quantization.ipynb` för att förstå hur kvantisering kan förbättra prestanda.

**Viktiga begrepp**:
- **Latens**: Inference-tid (kritiskt för edge)
- **Warmup**: Förbereder modellen för mätning
- **p50/p95**: Percentiler för latens-distribution
- **Varians**: Konsistens i prestanda
